In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import time
import math
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
from torch.utils.data import Dataset,RandomSampler
import operator
import numpy as np
%matplotlib inline

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PAD_token = 0
SOS_token = 1
EOS_token = 2
vocab_size = 85000
hidden_size = 256
# emb_size = 256
MAX_LENGTH_1 = 100 # since 99% source sentence is <= 100
# MAX_LENGTH_1 = max(len(pair[0].split(" ")) for pair in pairs)
# MAX_LENGTH_2 = max(len(pair[1].split(" ")) for pair in pairs)
dropout_p = 0.1
teacher_forcing_ratio = 0.5
BATCH_SIZE = 32

In [3]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [4]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"PAD", 1: "SOS", 2: "EOS",3:"UNK"}
        self.n_words = 4  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
        
            
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    s = unicodeToAscii(s.strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s


def readLangs(lang1, lang2):
    print("Reading lines...")

    # Read the file and split into lines
    lang1_lines = open('data/iwslt-zh-en/train.tok.zh', encoding = 'utf-8').read().\
                    strip().split('\n')
    lang2_lines = open('data/iwslt-zh-en/train.tok.en', encoding = 'utf-8').read().\
                    strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [(lang1_lines[i] +',' + lang2_lines[i]).split(',',1) for i in range(len(lang1_lines))]

    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

def prepareData(lang1, lang2):
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


def build_topwordVocab(lang, vocab_size):
    print("Build vocabulary by top {} frequent word...".format(vocab_size))
    sorted_word2Count = sorted(lang.word2count.items(),
        key=operator.itemgetter(1),
        reverse=True)
    sorted_words = [x[0] for x in sorted_word2Count[:vocab_size]]
    
    
    lang.index2word = {}
    lang.index2word[0] = "PAD"
    lang.index2word[1] = "SOS"
    lang.index2word[2] = "EOS"
    lang.index2word[3] = "UNK"
    
    for ind, word in enumerate(sorted_words):
            lang.index2word[ind + 4] = word
            

    lang.word2index = {}
    for ind, word in enumerate(sorted_words):
        lang.word2index[word] = ind + 4
    
    lang.n_words = len(lang.index2word)
    
    print(lang.name, lang.n_words)
    return lang

input_lang, output_lang, pairs = prepareData('eng', 'chi')

input_lang = build_topwordVocab(input_lang,vocab_size)
print(random.choice(pairs))

Reading lines...
Read 213376 sentence pairs
Counting words...
Counted words:
eng 88919
chi 69127
Build vocabulary by top 85000 frequent word...
eng 85004
['它 能 临近 96863 英尺   即 高 雷达 能 达到 的 最高 海拔    下午 四点 十二 十二分 二分   太阳 太阳神 阳神 号 已经 在 地球 大气 大气层 气层 的 百分 百分之 百分之九 百分之九十 百分之九十八 九十 九十八 十八 的 高度 了  ', 'Approaching a peak radar altitude of 96,863 feet , at 4 : 12 p.m. , Helios is standing on top of 98 percent of the Earth &apos;s atmosphere .']


In [5]:
sorted_word2Count = sorted(input_lang.word2count.items(),
    key=operator.itemgetter(1),
    reverse=True)

In [6]:
sorted_word2Count ###标点符号排第一 之后要改掉

[('', 1429891),
 ('的', 272264),
 ('我', 83174),
 ('是', 71290),
 ('我们', 68696),
 ('在', 55727),
 ('了', 52197),
 ('你', 40824),
 ('这', 36049),
 ('一个', 33590),
 ('他们', 25256),
 ('和', 24670),
 ('有', 23982),
 ('它', 18993),
 ('就', 18825),
 ('这个', 18294),
 ('他', 17734),
 ('会', 17319),
 ('可以', 17137),
 ('都', 16825),
 ('也', 15945),
 ('这些', 15501),
 ('人', 12741),
 ('说', 12586),
 ('做', 12416),
 ('什么', 12362),
 ('就是', 12343),
 ('中', 11481),
 ('因为', 11107),
 ('让', 10825),
 ('对', 10766),
 ('很', 10459),
 ('如果', 10402),
 ('但', 10311),
 ('上', 10295),
 ('没有', 9852),
 ('把', 9841),
 ('到', 9490),
 ('而', 9428),
 ('所以', 9324),
 ('现在', 9296),
 ('能', 9207),
 ('被', 8608),
 ('这样', 8510),
 ('知道', 8472),
 ('问题', 8444),
 ('想', 8259),
 ('来', 8238),
 ('要', 8109),
 ('将', 8049),
 ('但是', 8025),
 ('她', 7786),
 ('不是', 7708),
 ('当', 7708),
 ('去', 7602),
 ('那', 7585),
 ('你们', 7544),
 ('世界', 7446),
 ('从', 7432),
 ('自己', 7293),
 ('不', 7257),
 ('非常', 7100),
 ('它们', 7049),
 ('个', 6861),
 ('一些', 6844),
 ('开始', 6719),
 ('需要', 6586),

In [7]:
def indexesFromSentence(lang, sentence):
    idxs = []
    for word in sentence.split(' '):
        try:
            idxs.append(lang.word2index[word])
        except KeyError:
            idxs.append(3)  # 3 is the id of 'UNK'
    return idxs


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)



class VocabDataset(Dataset):
    def __init__(self, pairs):
#         pairs = [tensorsFromPair(pair) for pair in pairs]
#         self.source_sent_list = [i[0] for i in pairs]
#         self.target_sent_list = [i[1] for i in pairs]

        self.source_sent_list = [indexesFromSentence(input_lang,pair[0]) for pair in pairs]
        self.target_sent_list = [indexesFromSentence(output_lang,pair[1]) for pair in pairs]
        
    def __len__(self):
        return len(self.source_sent_list)
        
    def __getitem__(self, key):
        token1_idx = self.source_sent_list[key][:MAX_LENGTH_1]
        token2_idx = self.target_sent_list[key][:MAX_LENGTH_1]
        return [token1_idx,token2_idx, len(token1_idx), len(token2_idx)]

    
def Vocab_collate_func(batch):
    source_sent_list = []
    target_sent_list = []
    source_len_list = []
    target_len_list = []

    for datum in batch:   ### batch = sample
        source_len_list.append(datum[2])
        target_len_list.append(datum[3])

    # padding
    for datum in batch:
        
        # source sentence processing
        padded_source = np.pad(np.array(datum[0]), 
                                pad_width=((0,MAX_LENGTH_1-datum[2])),          ### 0代表左边没有pad,右边的值代表右边pad的个数
                                mode="constant", constant_values=PAD_token)
        source_sent_list.append(padded_source)
        
        # target sentence processing
        padded_target = np.pad(np.array(datum[1]), 
                                pad_width=((0,MAX_LENGTH_1-datum[3])),          ### 0代表左边没有pad,右边的值代表右边pad的个数
                                mode="constant", constant_values=PAD_token)
        target_sent_list.append(padded_target)
        
    return [torch.tensor(source_sent_list,device = device), 
            torch.tensor(target_sent_list,device = device),
            torch.LongTensor(source_len_list,device = device), 
            torch.LongTensor(target_len_list,device = device)]

train_dataset = VocabDataset(pairs)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=BATCH_SIZE,
                                           collate_fn=Vocab_collate_func,
                                           shuffle=True)

In [8]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, bidirectional=True) # embedding size = hidden size
        self.fc1 = nn.Linear(2*hidden_size, hidden_size)
    def initHidden(self,BATCH_SIZE):
        return torch.zeros(2, BATCH_SIZE, self.hidden_size, device=device) # return (2,1,hidden_size) 2 due to bidirection
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, BATCH_SIZE, -1)  # input is just one token at timpstep t
        output = embedded
        output, hidden = self.gru(output, hidden)  
        # (seq_len, batch, num_directions * hidden_size) and (num_layers * num_directions, batch, hidden_size)
        output = self.fc1(output)
        return output, hidden

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH_1):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, BATCH_SIZE, -1)
        embedded = self.dropout(embedded)
        
#         embedded: torch.Size([1, 32, 256])
#         hidden: torch.Size([1, 32, 256])
        
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)   
#         attn_weights:torch.Size([32, 100])
        attn_applied = torch.bmm(attn_weights.unsqueeze(1),
                                 encoder_outputs.transpose(0,1))
#         encoder_outputs: 100*32*512 attn_applied: 32*1*512
        
        output = torch.cat((embedded[0], attn_applied.transpose(0,1)[0]), 1)
        # output: 32*768
        output = self.attn_combine(output).unsqueeze(0)
        # output 1*32*256
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
#         output: torch.Size([32, 69126])
#         hidden: torch.Size([1, 32, 256])
        return output, hidden, attn_weights

In [9]:
# this is just one sentence input, could be batchlized 
def train(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, mask = None):
    encoder_hidden = encoder.initHidden(BATCH_SIZE)
    encoder_optimizer.zero_grad()  # zero out the accumulated gradient over mini-batch
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0) # length of source sentence
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(target_length, BATCH_SIZE, encoder.hidden_size, device=device) 
    # (seq_length, BATCH_SIZE,hidden_size*2) 2 due to bidirection

    loss = 0
    
    # feed-forward layer resulting encoder outputs, ei refers to each word token in input sentence
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)  
        # encoder_output: torch.Size([1, 32, 512]) encoder_hidden: torch.Size([2, 32, 256])
        encoder_outputs[ei] = encoder_output[0] 
    # change the shape of encoder output to fit into decoder 
    encoder_hidden = nn.Linear(2*hidden_size,hidden_size)(
        torch.cat((encoder_hidden[0],encoder_hidden[1]),dim = 1)).unsqueeze(0)
    
    decoder_input = torch.tensor([[SOS_token]*32], device=device)  # decoder_input: torch.Size([1, 32])
    # init decoder hidden 
    decoder_hidden = encoder_hidden
    
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            decoder_input = target_tensor[di]  # Teacher forcing
            temp_loss = criterion(decoder_output, target_tensor[di])
            loss += temp_loss * mask[di:di+1].float()  
            ave_loss = loss.sum()/BATCH_SIZE 
            
    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            # decoder_input: torch.Size([1, 32])
            # decoder_hidden: torch.Size([1, 32, 256]) 1 token * batch * hidden size
            # encoder_outputs: torch.Size([100, 32, 512])
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            # topv: 32*1
            # topi: 32*1

            decoder_input = topi.squeeze().detach()  # detach from history as input
            # decoder_input: 32
            # target_tensor: 100*32
            # decoder_output: 32*69127 
            temp_loss = criterion(decoder_output, target_tensor[di])
            loss += temp_loss * mask[di:di+1].float()
            # loss size 1*32
            ave_loss = loss.sum()/BATCH_SIZE  
            
    ave_loss.backward()
    
    
    encoder_optimizer.step()   # update parameters
    decoder_optimizer.step()

    return ave_loss.item() / target_length

In [10]:
def trainIters(encoder, decoder, n_iters, print_every=100, plot_every=100, learning_rate=0.01):
    start = time.time()

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss(reduce = False) ##!!!!!!!!!!1 这个loss是否要换成crossentropy

    for epoch in range(1, n_iters + 1):
        plot_losses = []
        print_loss_total = 0  # Reset every print_every
        plot_loss_total = 0  # Reset every plot_every
        for i, (input_sentences, target_sentences,len1,len2) in enumerate(train_loader): 
            input_tensor = input_sentences.transpose(0,1)   # 32*100 to 100*32
            target_tensor = target_sentences.transpose(0,1)
            mask = target_tensor.ge(1)   # 100 * 32
            loss = train(input_tensor, target_tensor, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion, mask = mask)
            print_loss_total += loss
            plot_loss_total += loss

            if i > 0 and i % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('Time: {}, Epoch: [{}/{}], Step: [{}/{}], Train Loss: {}'.format(
                    timeSince(start, i + 1/len(train_loader)), epoch, n_iters, i, 
                    len(train_loader),print_loss_avg))

            if i > 0 and i % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0
                
        print(plot_losses)
        showPlot(plot_losses)

In [11]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH_1):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden(BATCH_SIZE)

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS
        
        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]
    
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [12]:
encoder1 = EncoderRNN(input_lang.n_words,hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 3, print_every=1,plot_every=1)

torch.save(encoder1.state_dict(), "saved_model/encoder_hiddenSize{}".format(hidden_size))
torch.save(attn_decoder1.state_dict(), "saved_model/attn_decoder_hiddenSize{}".format(hidden_size))

/Users/hp/anaconda/lib/python3.5/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


KeyboardInterrupt: 

In [14]:
evaluateRandomly(encoder1, attn_decoder1, n=10)

> 我们 谈 的 不是 哈利 哈利波 波特 的 尾巴 下   它 是 在 左边  
= We &apos;re not talking about the Harry Potter end , right at the left side .


RuntimeError: input.size(-1) must be equal to input_size. Expected 256, got 8